In [14]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Convolution2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

batch_size = 64

nb_epoch = 100
nb_classes = 3
image_shape = [64, 64, 3]

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=image_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# save best model only 
# NOTE: needs validation_data in fit_generator or validation_split in ImageDataGenerator, which isn't available
# checkpointer = ModelCheckpoint(filepath='light_classifier_model.h5', save_best_only=True, verbose=1)

# reduce learning rate when loss doesn't improve much anymore (to pin-point the local minima)
reduce_lr = ReduceLROnPlateau(monitor='acc', epsilon=0.01, factor=0.4, patience=4, min_lr=0.001, verbose=1)

# stop when there was no real improvement anymore (even after reducing the learning rate several times)
early_stop = EarlyStopping(monitor='acc', min_delta=0.01, patience=12, verbose=1)


# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

datagen = ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, shear_range=0.05, zoom_range=.2,
                             fill_mode='nearest', rescale=1. / 255)
image_data_gen = datagen.flow_from_directory('Traffic_light', target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green'])

model.fit_generator(image_data_gen, nb_epoch=nb_epoch,
                    samples_per_epoch=image_data_gen.nb_sample, 
                    callbacks=[checkpointer, reduce_lr, early_stop])

model.save('light_classifier_model.h5')

Found 960 images belonging to 3 classes.
Epoch 1/100
960/960 [==============================] - 13s - loss: 1.5185 - acc: 0.4427    
Epoch 2/100
960/960 [==============================] - 12s - loss: 0.9985 - acc: 0.4677    
Epoch 3/100
960/960 [==============================] - 11s - loss: 0.9634 - acc: 0.5219    
Epoch 4/100
960/960 [==============================] - 11s - loss: 1.0153 - acc: 0.5563    
Epoch 5/100
960/960 [==============================] - 12s - loss: 0.8315 - acc: 0.6583    
Epoch 6/100
960/960 [==============================] - 13s - loss: 0.8048 - acc: 0.6427    
Epoch 7/100
960/960 [==============================] - 12s - loss: 0.7409 - acc: 0.6760    
Epoch 8/100
960/960 [==============================] - 12s - loss: 0.6838 - acc: 0.7167    
Epoch 9/100
960/960 [==============================] - 11s - loss: 0.6418 - acc: 0.7417    
Epoch 10/100
960/960 [==============================] - 11s - loss: 0.6058 - acc: 0.7708    
Epoch 11/100
960/960 [================